In [98]:
include("Create_Sample.jl")
include("MIP_SPLIT.jl")
include("Split.jl")
include("GA.jl")
include("Initial.jl")
include("Mutation.jl")
include("Crossover.jl")
include("Neighborhood.jl")
include("Neighborhood_intra.jl")
include("costs.jl")
include("Draw.jl")
include("Escape.jl")

Escape_local_optima (generic function with 1 method)

In [5]:
T = Read_TSPLIB_instance(:eil51, 1)
K = 4
n = size(T)[1]-2
demands = ones(Int, n)
W = 150
h = 0.3
popsize = (20,50)
k_tournament = 3
num_iter = 2000
Mutation_Chance = 0.0
counter_n = zeros(Int, 25)
counter_cr = zeros(Int, 7)
num_nei = 2
avg = 0.0
best = Inf
worst = 0.0
t1 = time() 
P = Chromosome[]

P, roullet = Perform_Genetic_Algorithm(T, demands,K, W, h, popsize, 
    k_tournament, num_iter, Mutation_Chance, num_nei, counter_n, counter_cr);

In [133]:
parent1 , parent2 = P[5], P[30]

(Chromosome([15, 49, 20, 28, 19, 34, 35, 2, 27, 30  …  33, 29, 9, 38, 32, 44, 14, 43, 16, 50], 142.4292111153954, 126.68386710127363, Tour[Tour([15, 49, 20, 28, 19, 34, 35, 2, 27, 30, 25, 7, 5], 142.4292111153954), Tour([21, 1, 4, 36, 41, 39, 18, 17, 26], 141.74787612046745), Tour([31, 45, 11, 46, 3, 40, 12, 24, 13, 23, 42, 6, 22, 47], 141.48016526425067), Tour([10, 37, 48, 8, 33, 29, 9, 38, 32, 44, 14, 43, 16, 50], 141.15637461556403)]), Chromosome([26, 47, 4, 8, 33, 29, 9, 38, 32, 44  …  41, 40, 39, 12, 24, 13, 23, 42, 6, 22], 177.36948351137173, 142.68937599111862, Tour[Tour([26, 47, 4, 8, 33, 29, 9, 38, 32, 44, 14, 43, 36, 16], 169.89061519942388), Tour([18, 17, 5, 10, 48, 50], 152.43321277370507), Tour([15, 49, 20, 28, 19, 34, 35, 2, 27, 30, 25, 7, 21, 1, 37, 31], 164.03297902942901), Tour([45, 11, 46, 3, 41, 40, 39, 12, 24, 13, 23, 42, 6, 22], 177.36948351137173)]))

In [157]:
include("Crossover.jl")
c = tour_crossover3(parent1, parent2, T, n)

4-element Vector{Tour}:
 Tour([15, 49, 20, 28, 19, 34, 35, 2, 27, 30, 25, 7, 21, 1, 37, 31], 164.032979029429)
 Tour([26, 47, 4, 36, 41, 39, 18, 17, 8, 33, 29, 9, 38, 32, 44, 14, 43, 16], 263.79734812967604)
 Tour([45, 11, 46, 3, 40, 12, 24, 13, 23, 42, 6, 22], 140.1044183646207)
 Tour([5, 10, 48, 50], 88.44695698759188)

In [158]:
nodes = Int[]
for tour in c
    println(find_tour_length(tour.Sequence, T), "   ", length(tour.Sequence))
    for i in tour.Sequence
        push!(nodes, i)
    end
end

println(length(nodes))

164.032979029429   16
263.79734812967604   18
140.10441836462073   12
88.44695698759186   4
50


In [120]:
outsiders

13-element Vector{Int64}:
  3
  6
 11
 12
 13
 22
 23
 24
 40
 42
 45
 46
 47

In [121]:
intersect(nodes, outsiders)

Int64[]

In [52]:
cities = [1]
index = 1
i = 1
seq = Int[]
seqs = Vector{Vector{Int}}()
while i <= cities[length(cities)]
    if i == cities[index]
        push!(seq, i)
        if i == cities[length(cities)]
            push!(seqs, seq)
        end
        i += 1
        index += 1
    else
        if length(seq)>0
            push!(seqs, seq)
            seq = Int[]
        end
        i += 1
    end
end
seqs

1-element Vector{Vector{Int64}}:
 [1]

In [50]:
tour = deepcopy(P[2].tours[1])

Tour([10, 48, 8, 33, 29, 9, 38, 32, 44, 14, 43, 16, 50], 141.15021864047716)

In [94]:
include("Crossover.jl")
t = deepcopy(P[2].tours[1])
Remove_cities_from_one_tour(t, [1,2,4,5,12,13], T, n)

117.6247269601199

In [96]:
t

Tour([10, 48, 8, 33, 29, 9, 38, 32, 44, 14, 43, 16, 50], 117.6247269601199)

In [95]:
t1 = [10, 48, 8, 33, 29, 9, 38, 32, 44, 14, 43, 16, 50]
t2 = [1,2,4,5,12,13]
deleteat!(t1, t2)
find_tour_length(t1, T)

117.62472696011989

In [80]:
seqs = [[1,2]]
tour = t.Sequence
cost = t.cost
for seq in seqs

    ns = length(seq)
    if seq[1] == 1
        if ns == 1
            println("A")
            println(cost)
            cost = cost - T[1, tour[1]+1] - T[tour[1]+1, tour[2]+1] + T[1, tour[2]+1]
            print(cost)
        else
            println("B")
            println(cost)
            cost = cost - T[1, tour[1]+1] - sum(T[tour[seq[i]]+1, tour[seq[i+1]]+1] for i=1:ns-1)- T[tour[seq[ns]]+1, tour[seq[ns]+1]+1] + T[1, tour[seq[ns]+1]+1]
            println(cost)
        end
    elseif seq[ns] == nt
        if ns == 1
            cost = cost - T[tour[nt]+1, n_nodes+2] - T[tour[nt-1]+1, tour[nt]+1] + T[tour[nt-1]+1, n_nodes+2]
        else
            cost = cost - T[tour[nt]+1, n_nodes+2] - sum(T[tour[seq[i]]+1, tour[seq[i+1]]+1] for i=1:ns-1) - T[tour[seq[1]-1]+1, tour[seq[1]]+1] + T[tour[seq[1]-1]+1, n_nodes+2] 
        end
    else
        if ns == 1
            cost = cost - T[tour[seq[1]-1]+1, tour[seq[1]]+1] - T[tour[seq[1]]+1, tour[seq[1]+1]+1] + T[tour[seq[1]-1]+1, tour[seq[1]+1]+1]
        else
            cost = cost - T[tour[seq[1]-1]+1, tour[seq[1]]+1] - sum(T[tour[seq[i]]+1, tour[seq[i+1]]+1] for i=1:ns-1) 
            - T[tour[seq[ns]]+1, tour[seq[ns]+1]+1] + T[tour[seq[1]-1]+1, tour[seq[ns]+1]+1]
        end
    end
end 

B
141.1501091684944
132.55355476801103


In [56]:
cost


141.14999969651163

In [62]:
t1 = [10, 48, 8, 33, 29, 9, 38, 32, 44, 14, 43, 16, 50]
find_tour_length(t1, T)

141.15021864047714

In [69]:
t1 = [8, 33, 29, 9, 38, 32, 44, 14, 43, 16, 50]
find_tour_length(t1, T)

132.5536642399938

In [79]:
seq = [1,2]
tour = [10, 48, 8, 33, 29, 9, 38, 32, 44, 14, 43, 16, 50]
println(T[1, tour[1]+1])
ns = length(seq)
println(sum(T[tour[seq[i]]+1, tour[seq[i+1]]+1] for i=1:ns-1))
println(T[tour[seq[ns]]+1, tour[seq[ns]+1]+1])
println(T[1, tour[seq[ns]+1]+1])
cost = 0
cost = cost - T[1, tour[1]+1] - sum(T[tour[seq[i]]+1, tour[seq[i+1]]+1] for i=1:ns-1)- T[tour[seq[ns]]+1, tour[seq[ns]+1]+1] + T[1, tour[seq[ns]+1]+1]

12.083045973594572
14.317821063276353
6.4031242374328485
24.20743687382041


-8.596554400483367

In [74]:
T[1,11]

12.083045973594572

In [75]:
T[11,49]

14.317821063276353

In [76]:
T[49,9]

6.4031242374328485

In [77]:
T[1,9]

24.20743687382041

In [130]:
sort(sample(1:10, 2, replace=false))

2-element Vector{Int64}:
 4
 6